<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Цель работы**

Обучить модель классифицировать комментарии на позитивные и негативные

**Описание данных**

Набор данных находится в файле /datasets/toxic_comments.csv.

Столбец text - текст комментария, а toxic — целевой признак.

**План работы**

- Изучить данные, выявить пропущенные значения
- Провести предобработку и анализ данных
- Разделить выборку на тренировочную, валидационную и тестовую
- Выбрать для каждой из трех моделей лучшие параметры
- Обучить три модели с выбранными параметрами и сравнить их по качеству
- Лучшую модель проверить на тестовой выборке
- Написать вывод

## Подготовка

In [26]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


**1.1 Изучение данных**

In [27]:
# загрузка данных 
data = pd.read_csv('/datasets/toxic_comments.csv')

In [28]:
# первые пять сторок и последние пять строк
display(data.head())
display(data.tail())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [29]:
# информация о данных 
data.info()
display(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


In [30]:
print(data.duplicated().sum())

0


In [31]:
# проверка балансировки классов
display(data['toxic'].value_counts())
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_ratio

0    143106
1     16186
Name: toxic, dtype: int64

8.841344371679229

В ходе изучения данных было получено следующее:
- данные состоят из 159292 строк и трех столбцов
- комментарии в столбце text на английском языке
- пропусков нет
- дубликатов нет
- столбец toxic - целевой признак
- в целевом признаке очевиден дисбаланс классов

**1.2 Подготовка данных**

In [32]:
# лемматизация и очистка текста
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_clear(text):
    text = text.lower()
    m = WordNetLemmatizer()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text_clear = " ".join(text.split())
    lemm_text = " ".join(m.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text_clear))
    return lemm_text

data['lemm_text'] = data['text'].apply(lemmatize_clear)
data = data.drop(['text'], axis=1)

# проверка
display(data.head(10))

,Unnamed: 0,toxic,lemm_text
0,0,0,explanation why the edits make under my userna...
1,1,0,d aww he match this background colour i m seem...
2,2,0,hey man i m really not try to edit war it s ju...
3,3,0,more i can t make any real suggestion on impro...
4,4,0,you sir be my hero any chance you remember wha...
5,5,0,congratulation from me a well use the tool wel...
6,6,1,cocksucker before you piss around on my work
7,7,0,your vandalism to the matt shirvington article...
8,8,0,sorry if the word nonsense be offensive to you...
9,9,0,alignment on this subject and which be contrar...


In [33]:
# отделим целевой признак
features = data.drop(['toxic'], axis=1)
target = data['toxic']

In [34]:
# разбиение данных на обучающую и тестовую выборки в отношении 90/10
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, 
                                                                          random_state=12345, stratify=target)

In [35]:
# tf_idf
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

tf_idf_train = count_tf_idf.fit_transform(features_train['lemm_text'].values)
tf_idf_test = count_tf_idf.transform(features_test['lemm_text'].values)

print(tf_idf_train.shape[0])
print(tf_idf_test.shape[0])

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


143362
15930


Данные готовы к работе

**Вывод по пункту 1**

В ходе изучения данных было получено следующее:
- данные состоят из 159292 строк и трех столбцов
- комментарии в столбце text на английском языке
- пропусков нет
- дубликатов нет
- столбец toxic - целевой признак
- в целевом признаке очевиден дисбаланс классов

В ходе подготовки данных было проделано следующее:
- проведена лемматизация и очистка текста
- отделен целевой признак
- данные разделены на обучающую и тестовую выборки
- проведена векторизация

## Обучение

Обучение выбранных моделей будем проводить с учетом дисбаланса классов с помощью метода балансировки по весу

**2.1 LogisticRegression**

In [36]:
dict_classes={0:1, 1:class_ratio}
model_log = LogisticRegression(max_iter=200)
hyperparams = [{'solver':['newton-cg', 'lbfgs', 'liblinear'], 'random_state':[12345], 'class_weight':[dict_classes]}]
score_log = GridSearchCV(model_log, hyperparams, scoring='f1', cv=5)
score_log.fit(tf_idf_train, target_train)
print("Best parameters set found on development set:")
print()
print(score_log.best_params_)
LR_best_params = score_log.best_params_
print()
print("Grid scores on development set:")
print()
means = score_log.cv_results_['mean_test_score']
for mean, params in zip(means, score_log.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
cv_f1_log = max(means)
print()
print('Best f1 LogisticRegression =', cv_f1_log)
best_model_log = score_log.best_estimator_

Best parameters set found on development set:

{'class_weight': {0: 1, 1: 8.841344371679229}, 'random_state': 12345, 'solver': 'lbfgs'}

Grid scores on development set:

0.752561 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'random_state': 12345, 'solver': 'newton-cg'}
0.752607 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'random_state': 12345, 'solver': 'lbfgs'}
0.752561 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'random_state': 12345, 'solver': 'liblinear'}

Best f1 LogisticRegression = 0.7526067813095907


**Вывод по пункту 2.1**

Лучшую точность показала модель с параметром solver='lbfgs'

**2.2 DecisionTreeClassifier**

In [37]:
model_tree = DecisionTreeClassifier()
max_depth_list = [x for x in range(1, 31)]
hyperparams = [{'max_depth':max_depth_list, 'random_state':[12345], 'class_weight':[dict_classes]}]
score_tree = GridSearchCV(model_tree, hyperparams, scoring='f1', cv=5)
score_tree.fit(tf_idf_train, target_train)
print("Best parameters set found on development set:")
print()
print(score_tree.best_params_)
print()
print("Grid scores on development set:")
print()
means = score_tree.cv_results_['mean_test_score']
for mean, params in zip(means, score_tree.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
cv_f1_tree = max(means)
print()
print('Best f1 DecisionTreeClassifier =', cv_f1_tree)
best_model_tree = score_tree.best_estimator_

Best parameters set found on development set:

{'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 30, 'random_state': 12345}

Grid scores on development set:

0.377228 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 1, 'random_state': 12345}
0.377110 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 2, 'random_state': 12345}
0.431348 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 3, 'random_state': 12345}
0.480103 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 4, 'random_state': 12345}
0.516295 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 5, 'random_state': 12345}
0.544476 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 6, 'random_state': 12345}
0.544654 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 7, 'random_state': 12345}
0.562920 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'max_depth': 8, 'random_state': 12345}
0.576810 for {'class_weight': {0: 1, 1: 

**Вывод по пункту 2.2**

С глубиной 30 модель DecisionTreeClassifier показала лучшую точность.

**2.3 SGDClassifier**

In [38]:
model_sgd = SGDClassifier()
hyperparams = [{'learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'],
                'loss':['hinge', 'log', 'modified_huber'],
                'eta0':[0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
                'random_state':[12345],
                'class_weight':[dict_classes]}]
score_sgd = GridSearchCV(model_sgd, hyperparams, scoring='f1', cv=5)
score_sgd.fit(tf_idf_train, target_train)
print("Best parameters set found on development set:")
print()
print(score_sgd.best_params_)
print()
print("Grid scores on development set:")
print()
means = score_sgd.cv_results_['mean_test_score']
for mean, params in zip(means, score_sgd.cv_results_['params']):
    print("%0.6f for %r"% (mean, params))
    print()
cv_f1_sgd = max(means)
print()
print('Best f1 SGDClassifier =', cv_f1_sgd)
best_model_sgd = score_sgd.best_estimator_

Best parameters set found on development set:

{'class_weight': {0: 1, 1: 8.841344371679229}, 'eta0': 0.01, 'learning_rate': 'adaptive', 'loss': 'modified_huber', 'random_state': 12345}

Grid scores on development set:

0.733610 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'eta0': 0.01, 'learning_rate': 'constant', 'loss': 'hinge', 'random_state': 12345}

0.713930 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'eta0': 0.01, 'learning_rate': 'constant', 'loss': 'log', 'random_state': 12345}

0.742908 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'eta0': 0.01, 'learning_rate': 'constant', 'loss': 'modified_huber', 'random_state': 12345}

0.735689 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'eta0': 0.01, 'learning_rate': 'optimal', 'loss': 'hinge', 'random_state': 12345}

0.713906 for {'class_weight': {0: 1, 1: 8.841344371679229}, 'eta0': 0.01, 'learning_rate': 'optimal', 'loss': 'log', 'random_state': 12345}

0.749819 for {'class_weight': {0: 1, 1: 8.841344371679229

**Вывод по пункту 2.3**

Лучшую точность показала модель с параметрами eta0=0.1, learning_rate='adaptive', loss='modified_huber'

**2.4 Анализ моделей**

**Сводная таблица результатов**

In [50]:
index = ['LogisticRegression','DecisionTreeClassifier', 'SGDClassifier']
data = {'f1':[cv_f1_log, cv_f1_tree, cv_f1_sgd]}
        
data_score = pd.DataFrame(data=data, index=index)
data_score['Рейтинг f1'] = (data_score['f1'])/(data_score['f1'].max())

data_score

,f1,Рейтинг f1
LogisticRegression,0.752607,0.999871
DecisionTreeClassifier,0.643736,0.855231
SGDClassifier,0.752704,1.000000


**Вывод по пункту 2**

В ходе обучения моделей было проделано следующее:
- подобрали лучшие параметры модели LogisticRegression(solver='newton-cg')
- подобрали лучшие параметры модели DecisionTreeClassifier(глубина 29)
- подобрали лучшие параметры модели SGDClassifier(eta0=0.1, learning_rate='adaptive', loss='modified_huber')
- результаты свели в таблицу
- по времени предсказания и метрике f1 лучшей оказалась модель SGDClassifier(eta0=0.1, learning_rate='adaptive', loss='modified_huber'), ее и будем проверять на тестовой выборке

## Выводы

In [51]:
%%time

# проверка финальной модели на тестовой выборке
predict_sgd = best_model_sgd.predict(tf_idf_test)

CPU times: user 187 µs, sys: 3.71 ms, total: 3.9 ms
Wall time: 2.83 ms


In [52]:
f1_sgd = f1_score(target_test, predict_sgd)
print('F1 SGDClassifier =', f1_sgd)

F1 SGDClassifier = 0.7523035230352304


На тестовой выборке финальная модель показала себя также хорошо

**Общий вывод**

В ходе изучения данных было получено следующее:
- данные состоят из 159292 строк и трех столбцов
- комментарии в столбце text на английском языке
- пропусков нет
- дубликатов нет
- столбец toxic - целевой признак
- в целевом признаке очевиден дисбаланс классов

В ходе подготовки данных было проделано следующее:
- проведена лемматизация и очистка текста
- отделен целевой признак
- данные разделены на обучающую и тестовую выборки
- проведена векторизация


В ходе обучения моделей было проделано следующее:
- подобрали лучшие параметры модели LogisticRegression(solver='newton-cg')
- подобрали лучшие параметры модели DecisionTreeClassifier(глубина 29)
- подобрали лучшие параметры модели SGDClassifier(eta0=0.1, learning_rate='adaptive', loss='modified_huber')
- резултаты свели в таблицу
- по времени предсказания и метрике f1 лучшей оказалась модель SGDClassifier(eta0=0.1, learning_rate='adaptive', loss='modified_huber'), ее и проверили на тестовой выборке

На тестовой выборке финальная модель показала себя также хорошо